In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten

# Carregando o Dataset
df = pd.read_csv('TuplasResults.csv')

# Codificação da variável alvo 'label'
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Tokenização e pad sequence para 'pt_stereotype'
max_words = 5000
max_sequence_length = 50

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['pt_stereotype'])
sequences = tokenizer.texts_to_sequences(df['pt_stereotype'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Divisão dos dados em conjuntos de treinamento e teste
X = padded_sequences
y = df['label_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Primeira Topologia: Simples LSTM
model1 = Sequential([
    Embedding(max_words, 32, input_length=max_sequence_length),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model1.summary()

# Segunda Topologia: LSTM + Dense
model2 = Sequential([
    Embedding(max_words, 32, input_length=max_sequence_length),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model2.summary()

# Terceira Topologia: Apenas camadas Dense
model3 = Sequential([
    Embedding(max_words, 32, input_length=max_sequence_length),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model3.summary()

# Treinamento dos modelos
model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
model3.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)



Epoch 1/50
6/6 [==============================] - 2s 117ms/step - loss: 0.6778 - accuracy: 0.7656 - val_loss: 0.6637 - val_accuracy: 0.7292
Epoch 2/50
6/6 [==============================] - 0s 31ms/step - loss: 0.6415 - accuracy: 0.7656 - val_loss: 0.6342 - val_accuracy: 0.7292
Epoch 3/50
6/6 [==============================] - 0s 33ms/step - loss: 0.6061 - accuracy: 0.7656 - val_loss: 0.6000 - val_accuracy: 0.7292
Epoch 4/50
6/6 [==============================] - 0s 35ms/step - loss: 0.5651 - accuracy: 0.7656 - val_loss: 0.5883 - val_accuracy: 0.7292
Epoch 5/50
6/6 [==============================] - 0s 35ms/step - loss: 0.5573 - accuracy: 0.7656 - val_loss: 0.6105 - val_accuracy: 0.7292
Epoch 6/50
6/6 [==============================] - 0s 34ms/step - loss: 0.5527 - accuracy: 0.7656 - val_loss: 0.5855 - val_accuracy: 0.7292
Epoch 7/50
6/6 [==============================] - 0s 34ms/step - loss: 0.5490 - accuracy: 0.7656 - val_loss: 0.5843 - val_accuracy: 0.7292
Epoch 8/50
6/6 [==========

In [37]:
# Avaliação dos modelos
print("Avaliação do Modelo 1")
model1.evaluate(X_test, y_test)


Avaliação do Modelo 1
2/2 [==============================] - 0s 8ms/step - loss: 0.9972 - accuracy: 0.7667


[0.9971500635147095, 0.7666666507720947]

In [38]:
print("Avaliação do Modelo 2")
model2.evaluate(X_test, y_test)

Avaliação do Modelo 2
2/2 [==============================] - 0s 9ms/step - loss: 0.9205 - accuracy: 0.8167


[0.920484185218811, 0.8166666626930237]

In [29]:
print("Avaliação do Modelo 3")
model3.evaluate(X_test, y_test)

Avaliação do Modelo 3
2/2 [==============================] - 0s 4ms/step - loss: 0.5245 - accuracy: 0.7500


[0.5244786143302917, 0.75]

In [39]:
# print("Resumo do Modelo 1")
# model1.summary()

# print("Resumo do Modelo 2")
# model2.summary()

print("Resumo do Modelo 3")
model3.summary()


Resumo do Modelo 3
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 50, 32)            160000    
                                                                 
 flatten_5 (Flatten)         (None, 1600)              0         
                                                                 
 dense_28 (Dense)            (None, 32)                51232     
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 211265 (825.25 KB)
Trainable params: 211265 (825.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# Texto real para teste
textos_reais = ["Os negros não são espertos o suficiente para serem cientistas"]

# Tokenização e padronização
sequencias_reais = tokenizer.texts_to_sequences(textos_reais)
sequencias_padded = pad_sequences(sequencias_reais, maxlen=max_sequence_length, padding='post', truncating='post')

# Previsões
predicao1 = model1.predict(sequencias_padded)
predicao2 = model2.predict(sequencias_padded)
predicao3 = model3.predict(sequencias_padded)

# Decodificação e exibição dos resultados
etiquetas_reais_1 = [label_encoder.inverse_transform([int(round(p[0]))])[0] for p in predicao1]
etiquetas_reais_2 = [label_encoder.inverse_transform([int(round(p[0]))])[0] for p in predicao2]
etiquetas_reais_3 = [label_encoder.inverse_transform([int(round(p[0]))])[0] for p in predicao3]

print("Previsões do Modelo 1:", predicao1)
print("Previsões do Modelo 2:", predicao2)
print("Previsões do Modelo 3:", predicao3)

print("Previsões do Modelo 1:", etiquetas_reais_1)
print("Previsões do Modelo 2:", etiquetas_reais_2)
print("Previsões do Modelo 3:", etiquetas_reais_3)


1/1 [==============================] - 0s 24ms/step
Previsões do Modelo 1: [[0.01562524]]
Previsões do Modelo 2: [[0.00678325]]
Previsões do Modelo 3: [[0.01705586]]
Previsões do Modelo 1: ['não-ofensivo']
Previsões do Modelo 2: ['não-ofensivo']
Previsões do Modelo 3: ['não-ofensivo']
